In [1]:
from datasets import load_dataset
from datasets.formatting.formatting import LazyBatch
ds = load_dataset("CharlyR/vtikz", "tikz", split="benchmark")
ds = ds.select_columns(["id","type","instruction","code","template_solution_code"])


/home/creux/Documents/AI/VIFagent/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
from collections import defaultdict
from notebooks.verifier_ds_creation.verifier_utils import (
    handle_choices,
    handle_def,
    handle_range,
    handle_rangei,
)
import re


def create_solutions(row: LazyBatch):
    all_created_template_codes: dict[str : list[str]] = defaultdict(list)
    for template_code in row["template_solution_code"]:
        template_code = template_code[
            0
        ]  # batch size always one, used for create more rows as output
        new_template_codes = handle_def(template_code)
        new_template_codes = [
            fin_templ_code
            for code in new_template_codes
            for fin_templ_code in handle_range(code)
        ]
        new_template_codes = [
            fin_templ_code
            for code in new_template_codes
            for fin_templ_code in handle_rangei(code)
        ]
        new_template_codes = [
            fin_templ_code
            for code in new_template_codes
            for fin_templ_code in handle_choices(code)
        ]
        all_created_template_codes[template_code] = new_template_codes

    new_rows = defaultdict(list)
    for template, created_template_codes in all_created_template_codes.items():
        for created_template_code in created_template_codes:
            for existing_col in set(row.keys()):
                new_rows[existing_col].append(row[existing_col][0])
            new_rows["original_template"].append(template)
            new_rows["solution"].append(created_template_code)
            new_rows["expected"].append(True)
    return new_rows

In [12]:
from datasets import Dataset

test_code = """
§choice([1,2],2)
qsdfqs
§choice([0,5],5)
qsd
"""
#test_ds = Dataset.from_dict({"template_solution_code":[[test_code]]})

expanded_ds = ds.select([0]).map(create_solutions,batched=True,batch_size=1,load_from_cache_file=False)

Map: 100%|██████████| 1/1 [00:00<00:00, 160.82 examples/s]


In [14]:
for row in expanded_ds.iter(1):
    print(row["original_template"][0])
    print("-----------------------------------------")
    print(row["solution"][0])
    break

\documentclass[tikz,border=5pt]{standalone}
\usepackage{pgfplots}
\pgfplotsset{/pgf/number format/use comma,compat=1.16}
\usepackage[T1]{fontenc}
\usepackage[utf8]{inputenc}
\usepackage{stanli} 
\usetikzlibrary{calc,intersections,patterns}

\begin{document}
\begin{tikzpicture}[scale=1]
  \draw (§rangei(1,0.5),§rangei(4.5,0.5)) node[below] {O};
  \draw[<->] (§rangei(2,0.5),§rangei(4.5,0.5)) node[right] {$x$}-|(§rangei(1,0.5),§rangei(5.5,0.5)) node[left]{$y$};
  \point{A}{2}{9};
  \point{B}{3}{4};
  \point{B'}{5}{5};
  \point{C}{0}{0};
  \point{D}{8}{1};
  \beam{2}{A}{B}[0][0];
  \beam{2}{B}{C}[0][0];
  \beam{2}{B}{D}[0][0];
  \beam{3}{A}{B'};
  \beam{3}{B'}{C};
  \beam{3}{B'}{D};
  \notation{4}{A}{B}[1];
  \notation{4}{C}{B}[2][0.6];
  \notation{4}{D}{B}[3];
  \support{3}{A}[180];
  \support{3}{C}[-45];
  \support{3}{D}[45];
  \hinge{1}{A}
  \hinge{1}{B}
  \hinge{1}{C}
  \hinge{1}{D}
  \hinge{1}{B'}
  \begin{scope}[color=red]
    \load{1}{B}[205][1.5];
    \notation{1}{1.3,3.5}{F}[cente

In [ ]:
#we'll see later about that
#ds.save_to_disk(f"dataset/.cache/{config_name}{split_name}")  # debug
#ds.push_to_hub("CharlyR/vtikz", config_name=subset, split=split_name)

##### some tests

In [ ]:
import re

test_code = """
§choice([0,1,2,12],5)
qsdfqs
§choice([0,5,10,65],5)
qsd
"""